In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
import os

ROOT_DIR = "../../"
sys.path.insert(0, os.path.abspath(ROOT_DIR + "/src"))
import visualization.visualization as vs
import scraping.preprocessing as pr

In [3]:
# Load system requirements data
df = pd.read_csv(ROOT_DIR + "data/sys_reqs/1-19-2021.sys_reqs.csv")

/home/chase0/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (24,34,38,40,41,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,92,93,94,95,96,97,98,100,101,102,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
app_ids = df.href.str.extract(r"[app|bundle]/(\d+)/")[0]
app_ids = app_ids.fillna(-1).astype(int)
df["app_id"] = app_ids

In [5]:
import requests
import time
import json

In [6]:
chunksize = 2000
unique_ids = df["app_id"].unique()
chunk_ids = np.array_split(unique_ids, len(unique_ids)/chunksize)

In [ ]:
%%capture output
save_dir = ROOT_DIR + "data/temp/steamspy/"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
chunk_fname = "steamspy_chunk_{}.csv"
start_chunk = 0
    
query = "https://steamspy.com/api.php?request=appdetails&appid={}"
chunk_num = start_chunk
failed_chunks = 0
failed_app_ids = []
curr_query = ''
for chunk in chunk_ids[start_chunk:]:
    outputs = {}
    for app_id in chunk:
        time.sleep(1)
        print(' '*len(curr_query), end='\r')
        curr_query = query.format(app_id)
        print("Fetching {}".format(curr_query), end='\r')
        
        out = requests.get(curr_query)
        if not out.ok:
            print(' '*len(curr_query), end='\r')
            print("Could not fetch details for app {}".format(app_id))
            failed_app_ids.append(app_id)
            outputs[app_id] = None
            continue
            
        content = json.loads(out.content)
        if content.get("name") is None:
            print(' '*len(curr_query), end='\r')
            print("No details found for app {}".format(app_id))
            failed_app_ids.append(app_id)
            outputs[app_id] = None
            continue
            
        outputs[app_id] = content
    try:
        data = pd.DataFrame(outputs).transpose()
        data.to_csv(save_dir + chunk_fname.format(chunk_num), index=False)
    except:
        print(' '*len(curr_query), end='\r')
        print("Chunk {} failed!".format(chunk_num))
        failed_chunks += 1
    chunk_num += 1

In [ ]:
log_dir = ROOT_DIR + "data/temp/logs/"
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
with open(log_dir + "output.txt", 'w') as f:
    f.write(output.stdout)

In [ ]:
# This will show capture all interim outputs from the above cell and allow access to them after running
output.show()

In [60]:
final_df = pd.read_csv(save_dir + os.listdir(save_dir)[0]).transpose()
for fname in os.listdir(save_dir)[1:]:
    new_df = pd.read_csv(save_dir + fname)#.transpose()
    final_df = final_df.append(new_df)
    
final_df = final_df.rename(dict(zip(range(20), data.index)), axis=1).reset_index(drop=True)

In [65]:
final_df.to_csv(ROOT_DIR + "data/steamspy.csv", index=False)